In [1]:
import random
import varseek as vk
import os
# from tqdm import tqdm
# tqdm.pandas()

varseek_directory = os.path.dirname(os.path.abspath(""))

In [2]:
mutant_sequencing_depth = 256
wt_sequencing_depth = 0
condition = ['variant_type:equal=substitution']

number_of_variants_to_sample_per_condition = 10_000
read_length = 150
strand = None  # None for strand-agnostic (randomly-selected), "f" for forward, "r" for reverse, "both" for both - make sure this matches the reference genome (vk build command) - strand = True -> "f" or "r" here; strand = False -> None or "both" here - note that the strand is randomly selected per *transcript*, such that all drawn reads will come from the same strand no matter what
add_noise_sequencing_error=True
add_noise_base_quality=False
error_rate=0.0001  # only if add_noise_sequencing_error=True
error_distribution=(0.85, 0.1, 0.05)  # sub, del, ins  # only if add_noise_sequencing_error=True
max_errors=float("inf")  # only if add_noise_sequencing_error=True
seq_id_column="seq_ID"
var_column="mutation_cdna"
threads = 16

k = 51
w = 47

# Paths
vk_ref_out_dir = os.path.join(varseek_directory, "data", "vk_ref_out")
reference_out_dir = os.path.join(varseek_directory, "data", "reference")
sequences = os.path.join(reference_out_dir, "ensembl_grch37_release93", "Homo_sapiens.GRCh37.cdna.all.fa")
reference_genome_fasta = os.path.join(reference_out_dir, "ensembl_grch37_release93", "Homo_sapiens.GRCh37.dna.primary_assembly.fa")
out_dir_notebook = os.path.join(varseek_directory, "data", "synthetic_reads_test_easy")  #!!! change for each run
read_fq_path = os.path.join(out_dir_notebook, "synthetic_reads.fq")

mutation_metadata_df = os.path.join(vk_ref_out_dir, "variants_updated_exploded_filtered.csv")
read_df_out = f"{out_dir_notebook}/synthetic_reads.csv"
mutation_metadata_df_out = f"{out_dir_notebook}/mutation_metadata_df_updated_vk_info_exploded_with_synthetic_read_info.csv"

In [ ]:
if not os.path.exists(read_fq_path):
    simulated_df_dict = vk.sim(
        variants=mutation_metadata_df,
        number_of_variants_to_sample=number_of_variants_to_sample_per_condition,
        strand=strand,
        number_of_reads_per_variant_alt=mutant_sequencing_depth,
        number_of_reads_per_variant_ref=wt_sequencing_depth,
        k=k,
        w=w,
        read_length=read_length,
        seed=None,  # no need to pass in since it's set globally
        add_noise_sequencing_error=add_noise_sequencing_error,
        error_rate=error_rate,
        error_distribution=error_distribution,
        max_errors=max_errors,
        with_replacement=False,  # set to True if wanting to have some reads that might be identical
        sequences=sequences,
        seq_id_column=seq_id_column,
        var_column=var_column,
        var_id_column="header",
        variant_type_column="vcrs_variant_type",
        reference_out_dir=reference_out_dir,
        filters=condition,
        out=out_dir_notebook,
        reads_fastq_out=read_fq_path,
        reads_csv_out=read_df_out,
        variants_updated_csv_out=mutation_metadata_df_out,
    )

/home/jmrich/Desktop/varseek/varseek/varseek_sim.py:305: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  variants = pd.read_csv(variants)
12:15:06 - INFO - cannot find mutant sequence read parent
12:15:06 - INFO - running varseek build
12:15:06 - WARNING - If running a workflow with vk ref or kb ref, k should be an odd number between 1 and 63. Got k=147.
12:15:06 - INFO - Using COSMIC email from COSMIC_EMAIL environment variable: jmrich@caltech.edu
12:15:06 - INFO - Using COSMIC password from COSMIC_PASSWORD environment variable
12:15:54 - INFO - Using var_id_column 'header' as the variant header column.
12:20:56 - INFO - Removed 204 variant-containing reference sequences with length less than 150...
12:20:56 - WARNING - 
        5372870 variants correctly recorded (100.00%)
        204 variants removed (0.00%)
          0 variants missing seq_id or var_column (0.000%)
          0 entries removed due to having a duplicate entry 

**number_variants = 10_000**

number_variants x mutant_sequencing_depth = number_of_reads
10_000 x 256 = 2_560_000

number_of_reads + 1 (header row) = total_df_lines
2_560_000 + 1 = 2_560_001

number_of_reads x 4 (4 lines per read) = total_fastq_lines
2_560_000 x 4 = 10_240_000

In [4]:
!wc -l {read_df_out}
!wc -l {read_fq_path}

2560001 /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/synthetic_reads.csv
10240000 /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/synthetic_reads.fq


In [19]:
print("Haplotypecaller, same strand")
# !python scripts/run_gatk_haplotypecaller_for_benchmarking.py --synthetic_read_fastq /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/synthetic_reads.fq --skip_accuracy_analysis --out /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/haplotypecaller --reference_genome_fasta /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --reference_genome_gtf /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.87.gtf --star_genome_dir data/reference/ensembl_grch37_release93/star_reference --java /home/jmrich/opt/jdk-17.0.12+7/bin/java --picard_jar /home/jmrich/opt/picard.jar --threads 16 --genomes1000_vcf /data/reference/ensembl_grch37_release93/1000GENOMES-phase_3.vcf --disable_tool_default_read_filters --tmp_dir /data/tmp2

print(f"Number of reads in STAR BAM (out of {number_of_variants_to_sample_per_condition * mutant_sequencing_depth})")
!samtools view -c -F 4 /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/haplotypecaller/alignment/sample_Aligned.sortedByCoord.out.bam

print(f"Number of variants in unfiltered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/haplotypecaller/vcfs/haplotypecaller/haplotypecaller_output_unfiltered.g.vcf.gz | wc -l

print(f"Number of variants in filtered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/haplotypecaller/vcfs/haplotypecaller/haplotypecaller_output_filtered_applied.vcf.gz | wc -l

Haplotypecaller, same strand
Number of reads in STAR BAM (out of 2560000)
2590420
Number of variants in unfiltered VCF (out of 10000)
109
Number of variants in filtered VCF (out of 10000)
114


In [23]:
print("Mutect2, same strand")
# !python scripts/run_gatk_mutect2_for_benchmarking.py --synthetic_read_fastq /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/synthetic_reads.fq --skip_accuracy_analysis --out /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/mutect2 --reference_genome_fasta /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --reference_genome_gtf /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.87.gtf --star_genome_dir data/reference/ensembl_grch37_release93/star_reference --java /home/jmrich/opt/jdk-17.0.12+7/bin/java --picard_jar /home/jmrich/opt/picard.jar --threads 16 --genomes1000_vcf /data/reference/ensembl_grch37_release93/1000GENOMES-phase_3.vcf --disable_tool_default_read_filters --tmp_dir /data/tmp3

print(f"Number of reads in STAR BAM (out of {number_of_variants_to_sample_per_condition * mutant_sequencing_depth})")
!samtools view -c -F 4 /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/mutect2/alignment/sample_Aligned.sortedByCoord.out.bam

print(f"Number of variants in unfiltered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/mutect2/vcfs/mutect2/mutect2_output_unfiltered.g.vcf.gz | wc -l

print(f"Number of variants in filtered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK/mutect2/vcfs/mutect2/mutect2_output_filtered_applied.vcf.gz | wc -l

Mutect2, same strand
Number of reads in STAR BAM (out of 2560000)
2590420
Number of variants in unfiltered VCF (out of 10000)
129
Number of variants in filtered VCF (out of 10000)
157


In [7]:
strand = "random"
read_fq_path_mixed_strands = os.path.join(out_dir_notebook, "synthetic_reads_mixed_strands.fq")
read_df_out = os.path.join(out_dir_notebook, "synthetic_reads_mixed_strands.csv")
mutation_metadata_df_out = os.path.join(out_dir_notebook, "mutation_metadata_df_updated_vk_info_exploded_with_synthetic_read_info_mixed_strands.csv")

if not os.path.exists(read_fq_path_mixed_strands):
    simulated_df_dict = vk.sim(
        variants=mutation_metadata_df,
        number_of_variants_to_sample=number_of_variants_to_sample_per_condition,
        strand=strand,
        number_of_reads_per_variant_alt=mutant_sequencing_depth,
        number_of_reads_per_variant_ref=wt_sequencing_depth,
        k=k,
        w=w,
        read_length=read_length,
        seed=None,  # no need to pass in since it's set globally
        add_noise_sequencing_error=add_noise_sequencing_error,
        error_rate=error_rate,
        error_distribution=error_distribution,
        max_errors=max_errors,
        with_replacement=False,  # set to True if wanting to have some reads that might be identical
        sequences=sequences,
        seq_id_column=seq_id_column,
        var_column=var_column,
        var_id_column="header",
        variant_type_column="vcrs_variant_type",
        reference_out_dir=reference_out_dir,
        filters=condition,
        out=out_dir_notebook,
        reads_fastq_out=read_fq_path_mixed_strands,
        reads_csv_out=read_df_out,
        variants_updated_csv_out=mutation_metadata_df_out,
    )

/home/jmrich/Desktop/varseek/varseek/varseek_sim.py:305: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  variants = pd.read_csv(variants)
13:36:44 - INFO - cannot find mutant sequence read parent
/home/jmrich/Desktop/varseek/varseek/varseek_sim.py:361: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  sim_data_df = pd.read_csv(update_df_out)
13:40:06 - INFO - Initial variant report
13:40:06 - INFO - Number of total variants: 5373074; VCRSs: 5373074; unique variants: 5373074; merged variants: 0

13:40:06 - INFO - variant_type equal substitution
13:40:10 - INFO - Number of total variants: 5143917 (229157 filtered); VCRSs: 5143917 (229157 filtered); unique variants: 5143917 (229157 filtered); merged variants: 0 (0 filtered)

13:40:10 - INFO - mutant_sequence_read_parent is_not_null None
13:40:16 - INFO - Number of total variants: 5143740 (177 filtered); VCRSs: 5143740 (177 fil

In [21]:
print("Haplotypecaller, mixed strands")
# !python scripts/run_gatk_haplotypecaller_for_benchmarking.py --synthetic_read_fastq /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/synthetic_reads_mixed_strands.fq --skip_accuracy_analysis --out /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/haplotypecaller --reference_genome_fasta /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --reference_genome_gtf /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.87.gtf --star_genome_dir data/reference/ensembl_grch37_release93/star_reference --java /home/jmrich/opt/jdk-17.0.12+7/bin/java --picard_jar /home/jmrich/opt/picard.jar --threads 16 --genomes1000_vcf /data/reference/ensembl_grch37_release93/1000GENOMES-phase_3.vcf --disable_tool_default_read_filters --tmp_dir /data/tmp2

print(f"Number of reads in STAR BAM (out of {number_of_variants_to_sample_per_condition * mutant_sequencing_depth})")
!samtools view -c -F 4 /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/haplotypecaller/alignment/sample_Aligned.sortedByCoord.out.bam

print(f"Number of variants in unfiltered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/haplotypecaller/vcfs/haplotypecaller/haplotypecaller_output_unfiltered.g.vcf.gz | wc -l

print(f"Number of variants in filtered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/haplotypecaller/vcfs/haplotypecaller/haplotypecaller_output_filtered_applied.vcf.gz | wc -l

Haplotypecaller, mixed strands
Number of reads in STAR BAM (out of 2560000)
2589084
Number of variants in unfiltered VCF (out of 10000)
9961
Number of variants in filtered VCF (out of 10000)
9936


In [26]:
print("Mutect2, mixed strands")
# !python scripts/run_gatk_mutect2_for_benchmarking.py --synthetic_read_fastq /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/synthetic_reads_mixed_strands.fq --skip_accuracy_analysis --out /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/mutect2 --reference_genome_fasta /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --reference_genome_gtf /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.87.gtf --star_genome_dir data/reference/ensembl_grch37_release93/star_reference --java /home/jmrich/opt/jdk-17.0.12+7/bin/java --picard_jar /home/jmrich/opt/picard.jar --threads 16 --genomes1000_vcf /data/reference/ensembl_grch37_release93/1000GENOMES-phase_3.vcf --disable_tool_default_read_filters --tmp_dir /data/tmp3

print(f"Number of reads in STAR BAM (out of {number_of_variants_to_sample_per_condition * mutant_sequencing_depth})")
!samtools view -c -F 4 /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/mutect2/alignment/sample_Aligned.sortedByCoord.out.bam

print(f"Number of variants in unfiltered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/mutect2/vcfs/mutect2/mutect2_output_unfiltered.g.vcf.gz | wc -l

print(f"Number of variants in filtered VCF (out of {number_of_variants_to_sample_per_condition})")
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/synthetic_reads_test_easy/GATK_mixed_strands/mutect2/vcfs/mutect2/mutect2_output_filtered_applied.vcf.gz | wc -l

Mutect2, mixed strands
Number of reads in STAR BAM (out of 2560000)
2589084
Number of variants in unfiltered VCF (out of 10000)
9986
Number of variants in filtered VCF (out of 10000)
7791


In [ ]:
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/variant_simulation_alternative_tools/vcfs/haplotypecaller/haplotypecaller_output_unfiltered.g.vcf.gz | wc -l
!zcat /home/jmrich/Desktop/RLSRWP_2025/data/variant_simulation_alternative_tools/vcfs/haplotypecaller/haplotypecaller_output_filtered_applied.vcf.gz | wc -l

155780
127822


# Unrelated stuff

In [6]:
import importlib
import RLSRWP_2025.seq_utils
importlib.reload(RLSRWP_2025.seq_utils)
from RLSRWP_2025.seq_utils import compare_two_vcfs_with_hap_py

#!!! this one-liner should do everything below
# from RLSRWP_2025 import compare_two_vcfs_with_hap_py

ground_truth_vcf = "/data/simulated_data_output/synthetic_cosmic_simulated_reads.vcf.gz"
test_vcf = "/data/variant_simulation_alternative_tools3/vcfs/haplotypecaller/haplotypecaller_output_unfiltered.g.vcf.gz"  # "/data/variant_simulation_alternative_tools/vcfs/haplotypecaller/haplotypecaller_output_unfiltered.g.vcf.gz"
reference_fasta = "/data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa"
unique_mcrs_df_path = "/data/simulated_data_output/unique_mcrs_df.csv"
output_dir = "/home/jmrich/Desktop/RLSRWP_2025/happy_new"
package_name = "gatk_haplotypecaller"

dry_run = True
compare_two_vcfs_with_hap_py(ground_truth_vcf = ground_truth_vcf, test_vcf = test_vcf, reference_fasta = reference_fasta, output_dir = output_dir, unique_mcrs_df = unique_mcrs_df_path, package_name = package_name, dry_run = dry_run)

Dry run is true. Run the following command in the terminal, or set dry_run = False:
sudo docker run --rm -it -v /data/simulated_data_output:/data/simulated_data_output -v /data/variant_simulation_alternative_tools3/vcfs/haplotypecaller:/data/variant_simulation_alternative_tools3/vcfs/haplotypecaller -v /data/reference/ensembl_grch37_release93:/data/reference/ensembl_grch37_release93 -v /home/jmrich/Desktop/RLSRWP_2025/happy_new:/home/jmrich/Desktop/RLSRWP_2025/happy_new mgibio/hap.py:v0.3.12 /opt/hap.py/bin/hap.py -r /data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --engine=scmp-somatic -o /home/jmrich/Desktop/RLSRWP_2025/happy_new/happy /data/simulated_data_output/synthetic_cosmic_simulated_reads.vcf.gz /data/variant_simulation_alternative_tools3/vcfs/haplotypecaller/haplotypecaller_output_unfiltered.g.vcf.gz


In [1]:
import os
import shutil
import pandas as pd
import gzip
import pysam
from tqdm import tqdm
import varseek as vk

unique_mcrs_df_path = "/home/jmrich/Desktop/RLSRWP_2025/data/simulated_data_output/unique_mcrs_df.csv"
unique_mcrs_df_original_path = "/home/jmrich/Desktop/RLSRWP_2025/data/simulated_data_output/unique_mcrs_df_original.csv"
cosmic_df_with_vcf_info_path = "/home/jmrich/Desktop/RLSRWP_2025/data/reference/cosmic/CancerMutationCensus_AllData_Tsv_v101_GRCh37/CancerMutationCensus_AllData_v101_GRCh37_vcf_info_for_fig2.csv"

unique_mcrs_df = pd.read_csv(unique_mcrs_df_path, usecols=["vcrs_header"])
cosmic_df_with_vcf_info_df = pd.read_csv(cosmic_df_with_vcf_info_path)

/tmp/ipykernel_247813/731798.py:14: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  cosmic_df_with_vcf_info_df = pd.read_csv(cosmic_df_with_vcf_info_path)


- explode unique_mcrs_df['vcrs_header'] by semicolon
- left merge in cosmic_df_with_vcf_info_df into unique_mcrs_df by ID for cosmic_df_with_vcf_info_df and vcrs_header for unique_mcrs_df
- groupby unique_mcrs_df with column vcrs_id, putting values in a list for CHROM, POS, REF, ALT and taking the first value for vcrs_header_original
- take the set of each of CHROM, POS, REF, ALT
- filter out rows with length of any of these sets larger than 1
- take the first element of each set as the new entry

In [ ]:
# give each row in unique_mcrs_df a unique ID
unique_mcrs_df['vcrs_id'] = ['vcrs_' + str(i) for i in range(1, len(unique_mcrs_df) + 1)]
unique_mcrs_df[['vcrs_id', 'vcrs_header']].to_csv("/home/jmrich/Desktop/RLSRWP_2025/data/simulated_data_output/unique_mcrs_id_to_header_dict_for_vcf.csv", index=False)

# Step 1: Explode `vcrs_header` by semicolon
unique_mcrs_df['vcrs_header_original'] = unique_mcrs_df['vcrs_header']
unique_mcrs_df = unique_mcrs_df.assign(
    vcrs_header=unique_mcrs_df['vcrs_header'].str.split(';')
).explode('vcrs_header')

# Step 2: Left merge with cosmic_df_with_vcf_info_df on vcrs_header ↔ ID
merged_df = unique_mcrs_df.merge(
    cosmic_df_with_vcf_info_df,
    how='left',
    left_on='vcrs_header',
    right_on='ID'
)

# Step 3: Group by vcrs_id, aggregate lists and take first original header
grouped = merged_df.groupby('vcrs_id').agg({
    'CHROM': lambda x: list(x.dropna()),
    'POS': lambda x: list(x.dropna()),
    'REF': lambda x: list(x.dropna()),
    'ALT': lambda x: list(x.dropna()),
    'vcrs_header_original': 'first'
}).reset_index()

# Step 4: Convert each to a set
grouped['CHROM_set'] = grouped['CHROM'].apply(set)
grouped['POS_set'] = grouped['POS'].apply(set)
grouped['REF_set'] = grouped['REF'].apply(set)
grouped['ALT_set'] = grouped['ALT'].apply(set)

# Step 5: Filter rows where any set has length > 1
filtered = grouped[
    (grouped['CHROM_set'].apply(len) == 1) &
    (grouped['POS_set'].apply(len) == 1) &
    (grouped['REF_set'].apply(len) == 1) &
    (grouped['ALT_set'].apply(len) == 1)
].copy()

# Step 6: Take first (only) element of each set
filtered['CHROM'] = filtered['CHROM_set'].apply(lambda s: next(iter(s)))
filtered['POS']   = filtered['POS_set'].apply(lambda s: next(iter(s)))
filtered['REF']   = filtered['REF_set'].apply(lambda s: next(iter(s)))
filtered['ALT']   = filtered['ALT_set'].apply(lambda s: next(iter(s)))

filtered['POS'] = filtered['POS'].astype(int)

# Optional: Drop the intermediate set columns
filtered = filtered.drop(columns=['CHROM_set', 'POS_set', 'REF_set', 'ALT_set'])
# filtered = filtered.rename(columns={'vcrs_header_original': 'ID'})
filtered = filtered.rename(columns={'vcrs_id': 'ID'})

In [3]:
print((unique_mcrs_df['vcrs_header_original'].str.contains(';')).sum())  # number with semicolon
print(len(grouped) - len(filtered))  # number with different VCF info

81108
25807


In [9]:
cosmic_simulated_vcf_file_name = "synthetic_cosmic_simulated_reads.vcf.gz"
cosmic_simulated_vcf = os.path.join("/data", "simulated_data_output", cosmic_simulated_vcf_file_name)
reference_genome_fasta = "/data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa"
reference_genome_fasta_index = reference_genome_fasta.replace(".fa", ".fa.fai")

if not os.path.isfile(cosmic_simulated_vcf):
    unsorted_vcf = cosmic_simulated_vcf.replace(".vcf.gz", "_unsorted_without_sample.vcf")
    vk.utils.write_to_vcf(filtered, output_file = unsorted_vcf)

    # add FORMAT/SAMPLE
    output_vcf_path = cosmic_simulated_vcf.replace(".vcf", "_unsorted.vcf")

    # Open original VCF
    in_vcf = pysam.VariantFile(unsorted_vcf, "r")

    # Clone header and add FORMAT and sample
    new_header = in_vcf.header.copy()

    # Add missing contigs to new_header
    for rec in in_vcf.fetch():
        contig = rec.contig
        if contig not in new_header.contigs:
            new_header.contigs.add(contig, length=1_000_000)  # Dummy length

    # Reset the iterator
    in_vcf.close()
    in_vcf = pysam.VariantFile(unsorted_vcf, "r")

    new_header.formats.add("GT", 1, "String", "Genotype")
    new_header.add_sample("synthetic")

    # Open output VCF
    out_vcf = pysam.VariantFile(output_vcf_path, "w", header=new_header)

    # Write each record with dummy genotype
    for rec in in_vcf:
        # Create a new record to match the new header
        new_rec = out_vcf.new_record(contig=rec.contig,
                                    start=rec.start,
                                    stop=rec.stop,
                                    id=rec.id,
                                    qual=rec.qual,
                                    filter=rec.filter.keys(),
                                    alleles=rec.alleles,
                                    info=dict(rec.info))

        new_rec.samples["synthetic"]["GT"] = (0, 1)  # Set genotype
        out_vcf.write(new_rec)

    # Close files
    in_vcf.close()
    out_vcf.close()

    !bcftools sort -O z -o {cosmic_simulated_vcf} {output_vcf_path}

if not os.path.isfile(f"{cosmic_simulated_vcf}.tbi"):
    !bcftools index -t {cosmic_simulated_vcf}

In [13]:
happy_base_dir = "/home/jmrich/Desktop/RLSRWP_2025"
happy_output_dir_name = "happy"

os.makedirs(os.path.join(happy_base_dir, happy_output_dir_name), exist_ok=True)
cosmic_simulated_vcf_in_happy_base_dir = os.path.join(happy_base_dir, happy_output_dir_name, cosmic_simulated_vcf_file_name)
reference_genome_fasta_in_happy_base_dir = os.path.join(happy_base_dir, happy_output_dir_name, "Homo_sapiens.GRCh37.dna.primary_assembly.fa")
reference_genome_fasta_index_in_happy_base_dir = reference_genome_fasta_in_happy_base_dir.replace(".fa", ".fa.fai")
if not os.path.exists(cosmic_simulated_vcf_in_happy_base_dir):
    shutil.copyfile(cosmic_simulated_vcf, cosmic_simulated_vcf_in_happy_base_dir)
if not os.path.exists(reference_genome_fasta_in_happy_base_dir):
    shutil.copyfile(reference_genome_fasta, reference_genome_fasta_in_happy_base_dir)
    shutil.copyfile(reference_genome_fasta_index, reference_genome_fasta_index_in_happy_base_dir)

In [30]:
alternative_tool = "gatk_haplotypecaller6"
alternative_tool_vcf_file_name = "haplotypecaller_output_unfiltered.g.vcf.gz"
alternative_tool_vcf = f"/home/jmrich/Desktop/RLSRWP_2025/data/variant_simulation_alternative_tools/vcfs/haplotypecaller/{alternative_tool_vcf_file_name}"
alternative_tool_vcf_in_happy_base_dir = os.path.join(happy_base_dir, happy_output_dir_name, alternative_tool_vcf_file_name)

if not os.path.exists(alternative_tool_vcf_in_happy_base_dir):
    shutil.copyfile(alternative_tool_vcf, alternative_tool_vcf_in_happy_base_dir)

happy_out_dir = os.path.join(happy_base_dir, happy_output_dir_name, alternative_tool)
os.makedirs(happy_out_dir, exist_ok=True)
if not os.listdir(happy_out_dir):
    print(f"sudo docker run --rm -it -v {happy_base_dir}:/data mgibio/hap.py:v0.3.12 /opt/hap.py/bin/hap.py -r /data/{happy_output_dir_name}/Homo_sapiens.GRCh37.dna.primary_assembly.fa --engine=scmp-somatic -o /data/{happy_output_dir_name}/{alternative_tool}/happy /data/{happy_output_dir_name}/{cosmic_simulated_vcf_file_name} /data/{happy_output_dir_name}/{alternative_tool_vcf_file_name}")
    # !sudo docker run --rm -it -v {happy_base_dir}:/data mgibio/hap.py:v0.3.12 /opt/hap.py/bin/hap.py -r /data/{happy_output_dir_name}/{Homo_sapiens.GRCh37.dna.primary_assembly.fa} --engine=scmp-somatic -o /data/{happy_output_dir_name}/{alternative_tool}/happy /data/{happy_output_dir_name}/{cosmic_simulated_vcf_file_name} /data/{happy_output_dir_name}/{alternative_tool_vcf_file_name}

sudo docker run --rm -it -v /home/jmrich/Desktop/RLSRWP_2025:/data mgibio/hap.py:v0.3.12 /opt/hap.py/bin/hap.py -r /data/happy/Homo_sapiens.GRCh37.dna.primary_assembly.fa --engine=scmp-somatic -o /data/happy/gatk_haplotypecaller6/happy /data/happy/synthetic_cosmic_simulated_reads.vcf.gz /data/happy/haplotypecaller_output_unfiltered.g.vcf.gz


In [ ]:
print(f"sudo docker run --rm -it -v {happy_base_dir}:/data mgibio/hap.py:v0.3.12 /opt/hap.py/bin/hap.py -r /data/{happy_output_dir_name}/Homo_sapiens.GRCh37.dna.primary_assembly.fa --engine=scmp-somatic -o /data/{happy_output_dir_name}/{alternative_tool}/happy /data/{happy_output_dir_name}/{cosmic_simulated_vcf_file_name} /data/{happy_output_dir_name}/{alternative_tool_vcf_file_name}")

In [15]:
summary_csv_path = os.path.join(happy_out_dir, "happy.summary.csv")
summary_df = pd.read_csv(summary_csv_path)
summary_df = summary_df.loc[summary_df["Filter"] == "ALL"]

# Sum total TP, FP, FN across all rows
total_tp = summary_df["TRUTH.TP"].sum()
total_fp = summary_df["QUERY.FP"].sum()
total_fn = summary_df["TRUTH.FN"].sum()

# Store in dictionary
results = {
    "TP": int(total_tp),
    "FP": int(total_fp),
    "FN": int(total_fn),
}

# # Save to file
# with open("happy_counts.txt", "w") as f:
#     for k, v in results.items():
#         f.write(f"{k}: {v}\n")

# Also print if desired
print(results)

{'TP': 339, 'FP': 3, 'FN': 5303345}


In [40]:
unique_mcrs_df = pd.read_csv(unique_mcrs_df_original_path)

# Step 1: Load happy.vcf.gz (annotated VCF)
happy_vcf = pysam.VariantFile(os.path.join(happy_out_dir, "happy.vcf.gz"))

# Step 2: Load original VCF (with IDs)
orig_vcf = pysam.VariantFile(cosmic_simulated_vcf)

# Step 3: Build a lookup dictionary from original VCF
orig_lookup = {}
for rec in orig_vcf.fetch():
    key = (rec.contig, rec.pos, rec.ref, tuple(rec.alts))
    orig_lookup[key] = rec.id

# Step 4: Collect IDs based on TP and FN
detected_ids_to_query_dp = {}
undetected_ids = set()

for rec in happy_vcf.fetch():
    for sample in rec.samples.values():
        bd = sample.get("BD")
        key = (rec.contig, rec.pos, rec.ref, tuple(rec.alts))
        match_id = orig_lookup.get(key)
        if match_id and bd in {"TP", "FP"}:
            query_dp = rec.info.get("QUERY_DP")
            detected_ids_to_query_dp[match_id] = query_dp
        elif match_id and bd == "FN":
            undetected_ids.add(match_id)


# TP: in detected and in synthetic
unique_mcrs_df['TP_gatk_haplotypecaller'] = (
    unique_mcrs_df['vcrs_id'].isin(detected_ids_to_query_dp) &
    unique_mcrs_df['included_in_synthetic_reads_mutant']
)

# FN: not in detected but in synthetic
unique_mcrs_df['FN_gatk_haplotypecaller'] = (
    ~unique_mcrs_df['vcrs_id'].isin(detected_ids_to_query_dp) &
    unique_mcrs_df['included_in_synthetic_reads_mutant']
)

# FP: in detected but NOT in synthetic
unique_mcrs_df['FP_gatk_haplotypecaller'] = (
    unique_mcrs_df['vcrs_id'].isin(detected_ids_to_query_dp) &
    ~unique_mcrs_df['included_in_synthetic_reads_mutant']
)

# TN: not in undetected and NOT in synthetic
unique_mcrs_df['TN_gatk_haplotypecaller'] = (
    ~unique_mcrs_df['vcrs_id'].isin(detected_ids_to_query_dp) &
    ~unique_mcrs_df['included_in_synthetic_reads_mutant']
)

print(unique_mcrs_df['TP_gatk_haplotypecaller'].sum())
print(unique_mcrs_df['FN_gatk_haplotypecaller'].sum())
print(unique_mcrs_df['FP_gatk_haplotypecaller'].sum())
print(unique_mcrs_df['TN_gatk_haplotypecaller'].sum())

127002
254835
2722
4944932


### STOP HERE - the code below was just for testing

In [10]:
# get FP lines
fp_lines = []
with gzip.open(os.path.join(happy_out_dir, "happy.vcf.gz"), "rt") as f:
    total_lines = sum(1 for line in f if not line.startswith("#"))

with gzip.open(os.path.join(happy_out_dir, "happy.vcf.gz"), "rt") as f:  # "rt" = read text mode
    for line in tqdm(f, total=total_lines, desc="Scanning VCF for FPs"):
        if line.startswith("#"):
            continue
        fields = line.strip().split("\t")
        format_fields = fields[8].split(":")
        sample_fields = fields[9:]  # assume >=1 samples
        for sample in sample_fields:
            sample_values = sample.split(":")
            sample_dict = dict(zip(format_fields, sample_values))
            if sample_dict.get("BD") == "FP":
                fp_lines.append(line.strip())
                break  # no need to check more samples

# Now fp_lines is a list of VCF lines marked as FP

Scanning VCF for FPs: 5303862it [00:22, 234898.21it/s]                             


In [13]:
fp_lines

['1\t1564591\t.\tG\tA\t4858.06\t.\tBS=1564522\tGT:BD:BK:BI:BVT:BLT:QQ\t0/1:FN:am:ti:SNP:het:.\t1/1:FP:am:ti:SNP:homalt:4858.06',
 '1\t2433568\t.\tC\tT\t1812.06\t.\tBS=2433568\tGT:BD:BK:BI:BVT:BLT:QQ\t0/1:FN:am:ti:SNP:het:.\t1/1:FP:am:ti:SNP:homalt:1812.06',
 '1\t3160684\t.\tG\tA\t2021.06\t.\tBS=3160661\tGT:BD:BK:BI:BVT:BLT:QQ\t0/1:FN:am:ti:SNP:het:.\t1/1:FP:am:ti:SNP:homalt:2021.06',
 '1\t3547556\t.\tG\tA\t6116.06\t.\tBS=3547542\tGT:BD:BK:BI:BVT:BLT:QQ\t0/1:FN:am:ti:SNP:het:.\t1/1:FP:am:ti:SNP:homalt:6116.06',
 '1\t6445578\t.\tG\tA\t2083.06\t.\tBS=6445565\tGT:BD:BK:BI:BVT:BLT:QQ\t0/1:FN:am:ti:SNP:het:.\t1/1:FP:am:ti:SNP:homalt:2083.06',
 '1\t10529339\t.\tC\tG\t6168.06\t.\tBS=10529244\tGT:BD:BK:BI:BVT:BLT:QQ\t0/1:FN:am:tv:SNP:het:.\t1/1:FP:am:tv:SNP:homalt:6168.06',
 '1\t11015082\t.\tC\tA\t2615.06\t.\tBS=11015049\tGT:BD:BK:BI:BVT:BLT:QQ\t0/1:FN:am:tv:SNP:het:.\t1/1:FP:am:tv:SNP:homalt:2615.06',
 '1\t36755199\t.\tA\tG\t6782.06\t.\tBS=36754933\tGT:BD:BK:BI:BVT:BLT:QQ\t0/1:FN:am:ti:SNP:het

In [20]:
!zgrep -P '^\S+\t1564591\t' {cosmic_simulated_vcf_in_happy_base_dir}

1	1564591	vcrs_4897191	G	A	.	PASS	.	GT	0/1


In [25]:
summary_csv_path = '/home/jmrich/Desktop/RLSRWP_2025/happy/gatk_haplotypecaller3/happy.summary.csv'
summary_df = pd.read_csv(summary_csv_path)
summary_df = summary_df.loc[summary_df["Filter"] == "ALL"]

# Sum total TP, FP, FN across all rows
total_tp = summary_df["TRUTH.TP"].sum()
total_fp = summary_df["QUERY.FP"].sum()
total_fn = summary_df["TRUTH.FN"].sum()

# Store in dictionary
results = {
    "TP": int(total_tp),
    "FP": int(total_fp),
    "FN": int(total_fn),
}

# # Save to file
# with open("happy_counts.txt", "w") as f:
#     for k, v in results.items():
#         f.write(f"{k}: {v}\n")

# Also print if desired
print(results)

{'TP': 339, 'FP': 3, 'FN': 5303345}


In [26]:
happy_out_dir

'/home/jmrich/Desktop/RLSRWP_2025/happy/gatk_haplotypecaller'

In [28]:
# get FP lines
fp_lines2 = []
with gzip.open("/home/jmrich/Desktop/RLSRWP_2025/happy/gatk_haplotypecaller3/happy.vcf.gz", "rt") as f:
    total_lines = sum(1 for line in f if not line.startswith("#"))

with gzip.open("/home/jmrich/Desktop/RLSRWP_2025/happy/gatk_haplotypecaller3/happy.vcf.gz", "rt") as f:  # "rt" = read text mode
    for line in tqdm(f, total=total_lines, desc="Scanning VCF for FPs"):
        if line.startswith("#"):
            continue
        fields = line.strip().split("\t")
        format_fields = fields[8].split(":")
        sample_fields = fields[9:]  # assume >=1 samples
        for sample in sample_fields:
            sample_values = sample.split(":")
            sample_dict = dict(zip(format_fields, sample_values))
            if sample_dict.get("BD") == "FP":
                fp_lines2.append(line.strip())
                break  # no need to check more samples

# Now fp_lines2 is a list of VCF lines marked as FP

Scanning VCF for FPs: 5303820it [00:22, 230671.79it/s]                             


In [29]:
fp_lines2

['12\t21033899\t.\tA\tT\t5026.06\t.\tExcessHet=0;FS=0;MQ=59.88;QD=20.94;SOR=0.71;QUERY_DP=240;QUERY_GQ=99;QUERY_GT=4,4;DP=241;AF=1;MLEAC=2;MLEAF=1;QUERY_AD=0,240;QUERY_PL=5040,697,0;AN=1;AC=1\tGT:QQ:BD:BI:BVT:BLT\t./.:.:.:.:NOCALL:nocall\t./1:5026.06:FP:tv:SNP:halfcall',
 '9\t103261118\t.\tG\tA\t4161.06\t.\tExcessHet=0;FS=0;MQ=60;QD=21.45;SOR=0.823;QUERY_DP=194;QUERY_GQ=99;QUERY_GT=4,4;DP=194;AF=1;MLEAC=2;MLEAF=1;QUERY_AD=0,194;QUERY_PL=4175,565,0;AN=1;AC=1\tGT:QQ:BD:BI:BVT:BLT\t./.:.:.:.:NOCALL:nocall\t./1:4161.06:FP:ti:SNP:halfcall',
 '9\t114393784\t.\tG\tA\t6155.06\t.\tExcessHet=0;FS=0;MQ=60;QD=24.82;SOR=0.759;QUERY_DP=248;QUERY_GQ=99;QUERY_GT=4,4;DP=248;AF=1;MLEAC=2;MLEAF=1;QUERY_AD=0,248;QUERY_PL=6169,743,0;AN=1;AC=1\tGT:QQ:BD:BI:BVT:BLT\t./.:.:.:.:NOCALL:nocall\t./1:6155.06:FP:ti:SNP:halfcall']